# 使用tensorboard来检测模型误差(loss)与准确率
reference:<br> 
[1] https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html <br>
[2] https://pytorch.org/docs/stable/tensorboard.html#torch-utils-tensorboard <br>
PS：使用了新的数据集FashionMINST,如果下载太慢可以使用以前CIFAR-10的数据集,记得更改全连接层输入维度 <br>
参考[1]是pytorch的文档，[2]是官方给的教程，我这里选了一部分来使用并添加了准确率的检测，想使用writer的其他方法可以在[1]中找

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## 使用了FashionMINST数据集

In [ ]:
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

In [ ]:
# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')


## 导入SummaryWriter并制定路径
writer = SummaryWriter(logdir) <br>
在使用tensorboard时，进入目录运行：tensorboard --logdir . 即可

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/fashion_mnist_experiment_1')

## 可视化几张数据集的图片并写入到tensorboard
add_image(tag, img_tensor, global_step=None, walltime=None, dataformats='CHW') : Add image data to summary.

In [ ]:
# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        
dataiter = iter(trainloader)
images, labels = dataiter.next()

img_grid = torchvision.utils.make_grid(images)

matplotlib_imshow(img_grid, one_channel=True)

writer.add_image('four_fashion_mnist_images', img_grid)

## 定义模型与损失函数

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().to(device)
print(net)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 将模型结果图写入tensorboard
add_graph(model, input_to_model=None, verbose=False) <br>
注意第二个参数为输入，这里因为net已经在gpu中所以input也要放入gpu中

In [ ]:
images, labels = images.to(device), labels.to(device)
writer.add_graph(net, images)

## 训练模型并在训练过程中将误差与准确率写入tensorboard
add_scalar(tag, scalar_value, global_step=None, walltime=None) : Add scalar data to summary.

In [ ]:
running_loss = 0.0
for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...
            data_count = epoch * len(trainloader) + i

            # ...log the running loss
            writer.add_scalar('training loss',
                            running_loss / 1000,
                            data_count)
            
            # ...log the accuracy
            with torch.no_grad():
                accuracy = 0
                total = 0
                for test_data in testloader:
                    net.eval()

                    test_inputs, test_labels = test_data
                    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)

                    test_outputs = net(test_inputs)

                    _, predicted = torch.max(test_outputs.data, 1)
                    total += test_labels.size(0)
                    accuracy += (predicted == test_labels).sum()

                acc = 100. * accuracy / total

                writer.add_scalar('accuracy/test', acc,data_count+1)
            print('[{}] running loss : {} accuracy: {}'.format(data_count+1,running_loss / 1000,acc))
            
            running_loss = 0.0
print('Finished Training')

## 使用完记得关闭writer

In [ ]:
writer.close()